In [9]:
import gymnasium as gym
print(gym.registry["ALE/Breakout-v5"])

KeyError: 'ALE/Breakout-v5'

In [7]:
!/usr/bin/python3 -m pip uninstall gymnasium ale-py -y
!/usr/bin/python3 -m pip install "gymnasium[atari]" ale-py --no-cache-dir

Found existing installation: gymnasium 1.1.1
Uninstalling gymnasium-1.1.1:
  Successfully uninstalled gymnasium-1.1.1
Found existing installation: ale-py 0.10.2
Uninstalling ale-py-0.10.2:
  Successfully uninstalled ale-py-0.10.2
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 1.2 MB/s eta 0:00:00ta 0:00:01
